# Bayesian Linear Regression

Why not MLE? They overfit.

Why not MAP? the prior fits the overfitting, but no representation of uncertainty.
ex: fit data to something (financial data), then have to predict value for new x. How certain are you? MAP doesn't say

Bayesian approach gets you level of uncertainty. Optimize the appropriate loss function.

gives us $p(y \mid x, D) $

## Setup

given data $D = [[x_1, y_1], ... [x_n, y_n]]$

model: $y_1, ... y_n$ independent given $w, Y_i \sim N(w^Tx_i, a^{-1})$

$a$ is the precision $1/\sigma^2$


$w \sim N(\boldsymbol 0, b^{-1}I), b > 0$

w is $w = (w_1, w_2, ... w_d)$, each is independent, i.i.d.

Assume a, b are known. So $\theta = w$, the only unknown.



## To model nonlinearies in Xs


Can replace $x_i$ by $\Phi(x_i)$ for some basis function $\Phi(x_i) = (\Phi_1(x_1) ... \Phi_n(x_i))$


## Compute Posterior Distribution

First need likelihood: $P(D|\theta) = P(D|w) \propto exp(-a/2(y-Aw)^T(y-Aw))$

A is the design matrix:

$$A =\begin{bmatrix} x_1^T \\ \vdots \\ x_n^T\end{bmatrix}$$

$y = (y_1, ... y_n)^T$


Posterior is the prob of w given the data $P(w|D) \propto p(D|w) P(w)\, \, \,  (likelihood * prior )$

$\propto exp(-a/2(y-Aw)^T(y-Aw)-\frac b 2 w^tw)$

this is quadratic in w, so it is Gaussian:

$$a(y-Aw)^T(y-Aw) =  bw^Tw = a(y^Ty - 2w^TA^Ty + w^TA^TAw) + bw^Tw \\
= ay^Ty - 2sw^TA^Ty + w^T(aA^TA + bI)w 
$$

Want to make this look like a gaussian. Gaussian, in general has $(w - \mu)\Lambda (w-\mu) = w^T\Lambda w - 2w^T\Lambda \mu + const$ Lambda is the precision matrix (inverse of covariance).

$\Lambda = aA^TA + bI$

$$2sw^TA^Ty = w^T\Lambda\mu \\
aA^Ty = \Lambda \mu \\
\mu = a\Lambda^{-1}A^Ty \\
\Lambda = aA^TA + BI$$

Therefore, posterior of w given data is
$$p(w|D) = N(w, \mu, \Lambda^{-1}) \\
\mu = a\Lambda^{-1}A^Ty \\
\Lambda = aA^TA + bI$$

Aside, is precision matrix invertible? In form $B+cI$. Think of eigenvalues. $Bu =\lambda u, cIu = cu$

$(B+cI)u = Bu + cIu = \lambda u + cu = (\lambda + c)u$. If all eigenvalues are strictly positive, then it is invertible.


### MAP estimate of w

Given $p(w|D) = N(w, \mu, \Lambda^{-1})$, we can get MAP. MAP is maximum a posteriori, value of w that maximizes the posterior distribution. MAP for a Gaussian is just the mean (top of bell curve!)

so, $w_{MAP} = \mu = a(aA^TA + bI)^{-1}A^Ty = \boxed{(A^TA + \frac b a I)^{-1}A^Ty}$

Compare to MLE:

$w_{MLE} = (A^TA)^{-1}A^Ty = A^+ y$

Difference is the $\frac b a I$, which is the *regularization parameter*. this may not correspond to a proper prior, you may make it from an improper prior, so more general than Bayesian. 


If you go back to $\propto exp(-a/2(y-Aw)^T(y-Aw)-\frac b 2 w^tw)$, $\frac b 2 w^tw$ is the regularization term. You are regularizing according to the squared norm of $w$.

test




## Predictive Distribution

$p(y\mid x, D)$. This is a *new* y that corresponds to a new $x$. 

In practice, use linear Gaussian models. But, to calculate ourselves:

$p(y\mid x, D) = \int p(y \mid x, D, w)p(w|x,D) dw$

x is just notational, the x for the y, so

$p(y \mid x, D, w) = p(y \mid x, w)$

$p(w \mid x, D) = p(w \mid D)$

Goal is to set $= \int N(w \mid ...) g(y) dw = g(y) \propto N(y \mid ...)$

$$p(y\mid \mathbf x, D) = \int N(y \mid w^T\mathbf x, a^{-1})N(w \mid \mu, \Lambda^{-1}) dw \\
\propto \int exp(-\frac a 2 (y-w^T\mathbf x)^2) exp(-\frac 1 2 (w-\mu)^T\Lambda(w-\mu))\\
= \int exp(-a/2 (y^2 - 2(w^T\mathbf x)y + (w^T\mathbf x)^2) - \frac 1 2 (w^T\Lambda w - 2w^T\Lambda \mu + \mu^T\Lambda\mu))
$$

pull out 1/2, expression for exp is:

$ay^2 - 2w^T\mathbf xya + aw^T\mathbf{xx}^Tw + w^T\Lambda w - 2w^T\Lambda\mu$ (dropped constant $mu^T\Lambda\mu$)

$= w^T(a\mathbf{xx}^T + \Lambda)w -2w^T(\mathbf xya - \Lambda\mu) + ay^2$

Now want to turn this into form gaussian form $(w-m)^TL(w-m) = w^TLw - 2w^TLm +m^TLm$.


let $L = a\mathbf{xx}^T + \Lambda$

$Lm = \mathbf xya + \Lambda \mu$

$m = L^{-1}(ay\mathbf x + \Lambda \mu$ (assuming L inverse exists)

Plug in

$$= w^TLw - 2w^TLm + m^TLm - m^TLm + ay^2 \\
= (w-m)^TL(W-m) - m^TLm + ay^2$$


$p(y \mid x,D) \propto \int \exp(-\frac 1 2 (w-m)^TL(w-m)) \times \exp(frac 1 2 m^TLm - \frac 1 2 ay^2) dw \\
\propto \exp(\frac 1 2 m^TLm - \frac 1 2 ay^2)$. 

this is g(y). Now rearrange to look like a normal!

$ay^2 - m^TLm$

$$m^TLm = (ay\mathbf x + \Lambda\mu)^TL^{-1}LL^{-1}(ay\mathbf x + \Lambda\mu) \\
= ay\mathbf x^TL^{-1}(ay\mathbf x) + 2ay\mathbf x^TL^{-1}\Lambda \mu + \mu^TL^{-1}\Lambda \mu \\
= (a^2\mathbf x^TL^{-1}x)y^2 + 2(a\mathbf x^TL^{-1}\Lambda \mu) y + const $$

So,

$$ay^2 - m^TLm = (a - a^2\mathbf x^TL^{-1}x)y^2 - 2(a\mathbf x^TL^{-1}\Lambda \mu) y +c $$

let $\lambda = a(1-ax^TL^{-1}x)$

$u = \frac 1 \lambda ax^TL^{-1}\Lambda \mu$

$$ p(y \mid \mathbf x, D) \propto \exp(- \frac \lambda 2(y-u)^2)$$


Can clean up:

$\boxed{u = \mu^Tx \\  \\
\frac 1 \lambda = \frac 1 a + \mathbf x^T\Lambda^{-1} \mathbf x \\
p(y \mid \mathbf x,D) = N(y \mid u, \frac 1 \lambda)}$

So, it is a Gaussian!!!!




Show value for $\lambda$:

$$L^{-1} = (axx^T + \Lambda)^{-1}$$

Use Sherman Morrison Theorem.

$$L^{-1} = \Lambda^{-1} - \frac{\Lambda^{-1}axx^T\Lambda^{-1}}{1+ax^T\Lambda^{-1}x}$$

$$x^T\Lambda^{-1}x = \alpha - \frac{a\alpha^2}{1+a\alpha} = \frac{\alpha + a\alpha^2 - a\alpha^2}{1+a\alpha}$$

$$\lambda = a(1-a\frac{a\alpha}{1+a\alpha}) = \frac{a}{1+a\alpha}$$



$$\mu^Tx = \frac 1 \lambda ax^TL^{-1}\Lambda \mu = \mu^T \frac 1 \lambda ax^TL^{-1}\Lambda
$$

$$\mathbf x = \frac 1 \lambda ax^TL^{-1}\Lambda$$

$$L \Lambda^{-1}x = \frac 1 \lambda a \mathbf x\\
(axx^T + \Lambda)\Lambda^{-1}x = axx^T\Lambda^{-1}x + x = (a\alpha + 1)x$$

$$\frac 1 \lambda = \frac{a\alpha + 1}{\alpha}$$